In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Read JSON").getOrCreate()

In [0]:
df_json=spark.read.json("/Volumes/workspace/practice/my_volume/employees_json/")
df_json.show()
df_json.printSchema()

+----------+-----+---------+----------+---------------+--------+-------+
|Department|EmpID|FirstName|  HireDate|       JobTitle|LastName| Salary|
+----------+-----+---------+----------+---------------+--------+-------+
|   Finance|   38|  First38|2015-02-08|        Analyst|  Last38|59000.0|
|     Sales|   39|  First39|2015-02-09|Sales Executive|  Last39|59500.0|
|        HR|   40|  First40|2015-02-10|      Recruiter|  Last40|60000.0|
|        IT|   41|  First41|2015-02-11|      Developer|  Last41|60500.0|
|   Finance|   42|  First42|2015-02-12|        Analyst|  Last42|61000.0|
|     Sales|   43|  First43|2015-02-13|Sales Executive|  Last43|61500.0|
|        HR|   44|  First44|2015-02-14|      Recruiter|  Last44|62000.0|
|        IT|   45|  First45|2015-02-15|      Developer|  Last45|62500.0|
|   Finance|   46|  First46|2015-02-16|        Analyst|  Last46|63000.0|
|     Sales|   47|  First47|2015-02-17|Sales Executive|  Last47|63500.0|
|        HR|   48|  First48|2015-02-18|      Recrui

In [0]:
df_json.write.mode("overwrite").saveAsTable("workspace.practice.employees_json")

In [0]:
# Multiline JSON

In [0]:
df_json=spark.read.option("multiline","true") .json("/Volumes/workspace/practice/my_volume/json_file.json")
df_json.show()

+---+------------+---+-------+------+
|age|        city| id|   name|salary|
+---+------------+---+-------+------+
| 28|    New York|  1|  Alice| 70000|
| 34| Los Angeles|  2|    Bob| 85000|
| 25|     Chicago|  3|Charlie| 62000|
| 45|     Houston|  4|  David| 95000|
| 30|     Phoenix|  5|    Eva| 72000|
| 38|Philadelphia|  6|  Frank| 88000|
| 27| San Antonio|  7|  Grace| 64000|
| 32|   San Diego|  8| Hannah| 79000|
| 29|      Dallas|  9|    Ian| 73000|
| 41|    San Jose| 10|  Julia| 91000|
+---+------------+---+-------+------+



In [0]:
# Nested JSON

In [0]:
df_nested = spark.read.option("multiline","true") .json("/Volumes/workspace/practice/my_volume/nested_json.json")
df_nested.show()

+-----------+-----------------+-----+--------------------+
|customer_id|            email| name|              orders|
+-----------+-----------------+-----+--------------------+
|          1|alice@example.com|Alice|[{2025-01-10, [{1...|
|          2|  bob@example.com|  Bob|[{2025-03-15, [{8...|
+-----------+-----------------+-----+--------------------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

# Define schema for nested JSON
item_schema = StructType([
    StructField("product", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("price", IntegerType(), True)
])

order_schema = StructType([
    StructField("order_id", IntegerType(), True),
    StructField("date", StringType(), True),
    StructField("items", ArrayType(item_schema), True)
])

customer_schema = StructType([
    StructField("customer_id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("email", StringType(), True),
    StructField("orders", ArrayType(order_schema), True)
])

# Read JSON file with schema
df = spark.read.option("multiLine", "true").schema(customer_schema).json("/Volumes/workspace/practice/my_volume/nested_json.json")

# Print schema
df.printSchema()
df.show()



root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- orders: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- order_id: integer (nullable = true)
 |    |    |-- date: string (nullable = true)
 |    |    |-- items: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- product: string (nullable = true)
 |    |    |    |    |-- quantity: integer (nullable = true)
 |    |    |    |    |-- price: integer (nullable = true)

+-----------+-----+-----------------+--------------------+
|customer_id| name|            email|              orders|
+-----------+-----+-----------------+--------------------+
|          1|Alice|alice@example.com|[{101, 2025-01-10...|
|          2|  Bob|  bob@example.com|[{201, 2025-03-15...|
+-----------+-----+-----------------+--------------------+



In [0]:
# Show raw nested data
df.show(truncate=False)




+-----------+-----+-----------------+------------------------------------------------------------------------------------------------+
|customer_id|name |email            |orders                                                                                          |
+-----------+-----+-----------------+------------------------------------------------------------------------------------------------+
|1          |Alice|alice@example.com|[{101, 2025-01-10, [{Laptop, 1, 1200}, {Mouse, 2, 25}]}, {102, 2025-02-05, [{Keyboard, 1, 75}]}]|
|2          |Bob  |bob@example.com  |[{201, 2025-03-15, [{Phone, 1, 800}, {Charger, 1, 20}]}]                                        |
+-----------+-----+-----------------+------------------------------------------------------------------------------------------------+



In [0]:
# Flatten orders
from pyspark.sql.functions import explode

orders_df = df.select("customer_id", "name","email", explode("orders").alias("order"))
orders_df.select("customer_id", "name","email", "order.order_id", "order.date").show()



+-----------+-----+-----------------+--------+----------+
|customer_id| name|            email|order_id|      date|
+-----------+-----+-----------------+--------+----------+
|          1|Alice|alice@example.com|     101|2025-01-10|
|          1|Alice|alice@example.com|     102|2025-02-05|
|          2|  Bob|  bob@example.com|     201|2025-03-15|
+-----------+-----+-----------------+--------+----------+



In [0]:
# Flatten items inside orders
items_df = orders_df.select("customer_id", "name","email", "order.order_id","order.date", explode("order.items").alias("item"))
item_df_final= items_df.select("customer_id", "name","email", "order_id", "date","item.product", "item.quantity", "item.price")
item_df_final.show()

+-----------+-----+-----------------+--------+----------+--------+--------+-----+
|customer_id| name|            email|order_id|      date| product|quantity|price|
+-----------+-----+-----------------+--------+----------+--------+--------+-----+
|          1|Alice|alice@example.com|     101|2025-01-10|  Laptop|       1| 1200|
|          1|Alice|alice@example.com|     101|2025-01-10|   Mouse|       2|   25|
|          1|Alice|alice@example.com|     102|2025-02-05|Keyboard|       1|   75|
|          2|  Bob|  bob@example.com|     201|2025-03-15|   Phone|       1|  800|
|          2|  Bob|  bob@example.com|     201|2025-03-15| Charger|       1|   20|
+-----------+-----+-----------------+--------+----------+--------+--------+-----+



In [0]:
item_df_final.write.mode("overwrite").saveAsTable("workspace.practice.customer_nested_json")
